In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from glob import glob
from PIL import Image
import random



In [ ]:
#EXPERIMENTAL BLOCK
from keras import backend as K
# jaccard similarity: the size of the intersection divided by the size of the union of two sets
def jaccard_index(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - ((intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0))

def dice_coeff(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return (2.0 * intersection + 1e-6) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1e-6)

def dice_loss(y_true, y_pred):
    coeff = dice_coeff(y_true, y_pred)
    return 1-coeff

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
data_gen_args = dict(
                     rescale=1./255,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1


image_generator = image_datagen.flow_from_directory(
    '../input/finalcrack/final crack/train/images',
    class_mode=None,
    color_mode='rgb',
    target_size=(224,224),
    batch_size=10,
    seed=seed)
mask_generator = mask_datagen.flow_from_directory(
    '../input/finalcrack/final crack/train/masks',
    class_mode=None,
    color_mode='grayscale',
    target_size=(224,224),
    batch_size=10,
    seed=seed)

train_generator = zip(image_generator, mask_generator)


In [ ]:
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1

image_generator = image_datagen.flow_from_directory(
    '../input/finalcrack/final crack/test/images',
    class_mode=None,
    color_mode='rgb',
    target_size=(224,224),
    batch_size=10,
    seed=seed)
mask_generator = mask_datagen.flow_from_directory(
    '../input/finalcrack/final crack/test/masks',
    class_mode=None,
    color_mode='grayscale',
    target_size=(224,224),
    batch_size=10,
    seed=seed)

test_generator = zip(image_generator, mask_generator)

In [ ]:
def double_conv_block(x,n_filters):
    
    x=layers.Conv2D(n_filters,3,padding="same",activation="relu",kernel_initializer="he_normal")(x)
    x=layers.Conv2D(n_filters,3,padding="same",activation="relu",kernel_initializer="he_normal")(x)
    return x


def upsample_block(x,conv_features,n_filters):
    
    
    
    x=layers.Conv2DTranspose(n_filters,3,2,padding="same")(x)
    x=layers.concatenate([x,conv_features])
    x=layers.Dropout(0.3)(x)
    x=double_conv_block(x,n_filters)
    return x

In [ ]:
def build_unet_model():

    inputs=layers.Input(shape=(224,224,3))
    encoder = keras.applications.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3),input_tensor=inputs)

    f1 = encoder.get_layer('block1_conv2').output
    f2 = encoder.get_layer('block2_conv2').output
    f3 = encoder.get_layer('block3_conv3').output
    f4 = encoder.get_layer('block4_conv3').output


    bottleneck = encoder.get_layer('block5_conv3').output


    u6=upsample_block(bottleneck,f4,512)
    u7=upsample_block(u6,f3,256)
    u8=upsample_block(u7,f2,128)
    u9=upsample_block(u8,f1,64)

    outputs=layers.Conv2D(1,1,padding="same",activation="sigmoid")(u9)
    print(outputs.shape)

    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model




In [ ]:
unet_model = build_unet_model()

ad=tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam")
                
model = build_unet_model()
model.compile(optimizer=ad,
                  loss=dice_loss,
                  metrics=['accuracy'] )
model_history=model.fit(train_generator,steps_per_epoch=300,epochs=10,validation_data=test_generator,validation_steps=30)
model.save('unet3')

In [ ]:
image = cv2.imread("../input/asdfasdf/crack_in_large_context_04.jpeg", cv2.IMREAD_COLOR)
image = cv2.resize(image, (224, 224),interpolation=cv2.INTER_AREA)
plt.subplot(1,2,1)
plt.imshow(image)
image = tf.cast(image, tf.float32) / 255.0
image = np.expand_dims(image,axis=0)
last = model.predict(image)[0,:,:,0]
print(last)
plt.subplot(1,2,2)
plt.imshow(last) 

In [ ]:
# b = 190
# for a in range(b,b+5):
#     plt.figure(figsize=(10,10))
#     plt.subplot(1,3,1)
#     plt.axis(False)
#     plt.title("Original Image", fontweight="bold")
    
#     plt.imshow(x[a])
#     plt.subplot(1,3,2)
#     plt.imshow(y[a])
#     plt.subplot(1,3,3)
#     plt.axis(False)
#     plt.title("Segmentation Mask", fontweight="bold")
#     img = np.expand_dims(x[a],axis=0)
#     last = unet_model.predict(img)[0,:,:,0]
#     plt.imshow(last)

In [ ]:
loss = model_history.history['loss']
print(loss)

val_loss = model_history .history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = model_history .history['accuracy']
val_acc = model_history .history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()